<a href="https://colab.research.google.com/github/AdnaneMOUZAOUI/Analyse-de-sentiments-machine-learning/blob/Reviews-Analysis/XGBoost_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install xgboost

In [ ]:
# Importer le fichier depuis le drive
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import nltk
import re
from sklearn.feature_selection import VarianceThreshold, SelectKBest, SelectFromModel, f_regression, mutual_info_regression, RFE, RFECV
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import GradientBoostingClassifier
import joblib
# df=df.rename(columns={"nom_client":"client","date_achat":"date_commande","date_avis":"date","text_avis":"Text","titre_avis":"Titre","note_avis":"star"})

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    print("Drive non connecté")

try:
    df=pd.read_csv('/content/drive/MyDrive/Colab_supply_chain/Data/commentaires_prepares.csv')
except:
    df=pd.read_csv("commentaires_prepares.csv")


df=df.dropna(axis=0,how='any')
df=df.rename(columns={"Commentaire":"Text"})
df['Text'] = df['Text'].apply(lambda x: ' '.join(re.findall(r"'(\w+)'", x)))
display(df['Text'].head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


0    jamais reçu commande écris showroomprive clôtu...
1    commandé collier lun ok lautre tout celui comm...
2    commandé airpods reconditionnés dit excellent ...
3    produit acheté septembre retourné mois attends...
4    encore commande partiellement annulée dernier ...
Name: Text, dtype: object

In [ ]:
# Création d'une nouvelle variable "star_cluster" pour séparer le jeu de données en 2 segments,
# le segment des avis positifs (=1) et le segment des avis négatifs (=0)
df['Sentiment'] = df['star'].apply(lambda x: 1 if x >= 4 else 0)
df=df[["Text","Sentiment"]]
df["Text"].fillna('', inplace=True)
df

<ipython-input-4-e714f905be07>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Text"].fillna('', inplace=True)


,Text,Sentiment
0,jamais reçu commande écris showroomprive clôtu...,0
1,commandé collier lun ok lautre tout celui comm...,0
2,commandé airpods reconditionnés dit excellent ...,0
3,produit acheté septembre retourné mois attends...,0
4,encore commande partiellement annulée dernier ...,0
...,...,...
168892,satisfaite commande,1
168893,très bien très rapide satisfaite commande,1
168894,commandé mi octobre pouvoir recevoir invités f...,0
168895,lorsque vu toile site cela plu pensais queffec...,0


In [ ]:
# Séparer la variable explicative de la variable à prédire
X, y = df.Text, df.Sentiment

# Séparer le jeu de données en données d'entraînement et données test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Réduire la taille de l'échantillon
sample_size = int(0.5 * len(X_train)) #10% de la taille initiale
X_train = X_train.sample(n=sample_size, random_state=42)
y_train = y_train.loc[X_train.index]

In [ ]:
# Initialiser un objet vectorizer
vectorizer = CountVectorizer()

# Mettre à jour la valeur de X_train et X_test
X_train = vectorizer.fit_transform(X_train).todense()
X_test = vectorizer.transform(X_test).todense()

In [ ]:
import xgboost as xgb

# X_train = np.asarray(X_train)
# y_train = np.asarray(y_train)
# X_test = np.asarray(X_test)

# Créer le classificateur XGBoost
clf = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.5,
    max_depth=3,
    n_jobs=-1,
    )

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
#  Importer la classe classification_report
from sklearn.metrics import classification_report

# Calcul et affichage de classification_report
print( classification_report(y_test, y_pred) )

# Calcul et affichage de la matrice de confusion
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix

In [ ]:
comments = ["Incroyable service, je suis ravie",
            "Je ne recommande pas"]

tokenized_comments = vectorizer.transform(comments)

clf.predict(tokenized_comments.toarray())

In [ ]:
import joblib

# Sauvegarder le modèle
joblib.dump(clf, 'XGBoost1.pkl')


In [ ]:
import joblib
# Charger le modèle à partir du fichier
clf = joblib.load('XGBoost1.pkl')

# Utiliser le modèle chargé pour effectuer des prédictions
comments = ["Incroyable service, je suis ravie",
            "Je ne recommande pas"]

tokenized_comments = vectorizer.transform(comments)
clf.predict(tokenized_comments.toarray())